# install

In [16]:
from ctgan import CTGAN
import ctgan
ctgan.__version__

'0.6.0'

In [ ]:
# from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
# from sdv.tabular import CTGAN
import warnings
import joblib
import os
import torch
import random
import numpy as np
import glob
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from tqdm import tqdm
from scipy.signal import butter, lfilter

warnings.filterwarnings('ignore')

In [46]:
import numpy as np
import random
import os
import torch

def seed_everything(seed: int=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)               # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True      # type: ignore

''' sample '''
seed_everything(0)

In [47]:
model = CTGAN(verbose=True)
model.fit(sample, epochs=1) # discrete_cols, epochs=epochs)

Epoch 1, Loss G:  0.1015,Loss D:  0.0156


In [48]:
# check seed everything
out = model.sample(10)
out

,sample
0,2
1,5
2,2
3,1
4,1
5,5
6,4
7,6
8,4
9,4


# def

In [ ]:
def LPF(series, low, order=1):
   
    b, a = butter(
                  N = order,
                  Wn = low,
                  btype = 'low',
                  )
    lpf_series = lfilter(b, a, series)
    
    return lpf_series

def preprocessing_2(data):
    data['obs_time'] = data.index % 24  # 시간통일
    col_list = data.columns
    for i in range(0, len(col_list)):
        col = col_list[i]
        if '누적' in col:
            data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(
                col_list[i-1]).cumsum()
    return data

def filtering(df):

    f1 = df[df['co2관측치']>578]['Case'].unique().tolist()
    f2 = df[((df['내부습도관측치']<74.4)&(df['내부습도관측치']>73.6))|((df['내부습도관측치']<30)&(df['내부습도관측치']>27))|((df['내부습도관측치']<54.5)&(df['내부습도관측치']>54.4)) ]['Case'].unique().tolist()
    f3 = df[(df['일간누적분무량']<150)]['Case'].unique().tolist()
    
    d1 = df[(df['시간당분무량']>2100)]['Case'].unique().tolist()
    d2 = df[(df['내부온도관측치']>31.124)&(df['내부온도관측치']<31.126)]['Case'].unique().tolist()
    d3 = df[(df['co2관측치']<510.1)&(df['co2관측치']>=510)]['Case'].unique().tolist()
    d4 = df[(df['시간당백색광량']>3094.0)&(df['시간당백색광량']<3094.2)]['Case'].unique().tolist()
    d5 = df[(df['시간당백색광량']>13400)&(df['시간당백색광량']<13400.5472)]['Case'].unique().tolist()

    return_arr = list(set(f1)&set(f2)&set(f3))
    for i in set(d1+d2+d3+d4+d5):
        if i in return_arr :
            return_arr.remove(i)

    return return_arr

def makedatasets(input_path):
    print("###################################################################################################################")

    test = preprocessing_2(input_path) #input_path : 생성될 데이터셋 path
    print(test)
    test['05시내부온도관측치누적'] = 0
    test['19시내부온도관측치누적'] = 0
    test['23시내부온도관측치누적'] = 0
    print("###################################################################################################################")

    test['05시내부습도관측치누적'] = 0
    test['19시내부습도관측치누적'] = 0
    test['23시내부습도관측치누적'] = 0

    test['05시co2관측치누적'] = 0
    test['19시co2관측치누적'] = 0
    test['23시co2관측치누적'] = 0

    test['05시ec관측치누적'] = 0
    test['19시ec관측치누적'] = 0
    test['23시ec관측치누적'] = 0

    test['05시분무량누적'] = 0
    test['19시분무량누적'] = 0
    test['23시분무량누적'] = 0

    test['05시백색광누적'] = 0
    test['19시백색광누적'] = 0
    test['23시백색광누적'] = 0

    test['05시적색광누적'] = 0
    test['19시적색광누적'] = 0
    test['23시적색광누적'] = 0

    test['05시청색광누적'] = 0
    test['19시청색광누적'] = 0
    test['23시청색광누적'] = 0

    test['05시총광량누적'] = 0
    test['19시총광량누적'] = 0
    test['23시총광량누적'] = 0
    print("#############################################zxczxczxczxczxc####################################################")
    
    testfilter = filtering(test)

    test = test.drop(test.filter(regex='일간누적').columns, axis=1)

    test_new = test
    print("#############################################qqqqqqqqqqqqqqqqqq#######################################################")

    test['시간대'] = 0
    test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
    test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
    test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

    #########################################################교체#####################

    print("###################################################################################################################")
    test['측정될수분량2'] = 0
    print(test.shape)
    for i in range(22,len(test),24):
        s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 40] = s

    print("########################################################aaaa###########################################################")


    test['측정될수분량1'] = 0
    for i in range(22,len(test),24):
        s = test.iloc[i+1,6] * (test.iloc[i+1,3])
        test.iloc[i, 41] = s

    print("########################################################sssss###########################################################")


    test['측정될수분량3'] = 0
    for i in range(22,len(test),24):
        s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 42] = s

    ###################################################################################################################

    print("########################################################22222222###########################################################")
    test_x = test
    test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()

    print("####################################################433333334###############################################################")

    test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
    test_x.drop(['index'], axis = 1, inplace=True)


    t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부온도관측치'] = t

    t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t2.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부습도관측치'] = t2

    t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t3.drop(['level_1'], axis = 1, inplace = True)
    test_x['co2관측치'] = t3

    t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t4.drop(['level_1'], axis = 1, inplace = True)
    test_x['ec관측치'] = t4

    t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t5.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당분무량'] = t5

    t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t6.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당백색광량'] = t6

    t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t7.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당적색광량'] = t7

    t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t8.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당청색광량'] = t8

    t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t9.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당총광량'] = t9

    test_new_x = test_new
    test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()

    test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
    test_new_x.drop(['index'], axis = 1, inplace=True)

    print("####################################################44444###############################################################")

    case_list = test_x['Case'].unique()

    for c in case_list : 
        if c in testfilter :
            continue
        test_x.loc[(test_x['Case'] == c ), '측정될수분량2'] = 0
        test_x.loc[(test_x['Case'] == c ), '측정될수분량1'] = 0
        test_x.loc[(test_x['Case'] == c ), '측정될수분량3'] = 0

    for i in range(len(test_new_x)):
        #온도
        test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
        test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
        test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
        #습도
        test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
        test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
        test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
        #co2
        test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
        test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
        test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
        #ec
        test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
        test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
        test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
        #분무
        test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
        test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
        test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
        #백색
        test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
        test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
        test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
        #적색
        test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
        test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
        test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
        #청색
        test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
        test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
        test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
        #총광
        test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
        test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
        test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]

    print("#####################################################555555##############################################################")


    test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)

    test_x = test_new_x

    test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)


    test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
    test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
    test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
    test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
    test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
    test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
    test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
    test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
    test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3

    test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)


    test_x['ec_x_분무05'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
    test_x['ec_x_분무19'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
    test_x['ec_x_분무23'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
    test_x['ec_x_분무평균'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)

    test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
    test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
    test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
    test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])


    test_x['수분량합'] =  test_x['측정될수분량1'] + test_x['측정될수분량2'] + test_x['측정될수분량3']

    test_x['수분량합12'] =  test_x['측정될수분량1'] + test_x['측정될수분량2']

    test_x['수분량합13'] =  test_x['측정될수분량1'] + test_x['측정될수분량3']

    test_x['수분량합23'] =  test_x['측정될수분량2'] + test_x['측정될수분량3']

    test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)

    test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)

    test_x.drop(['측정될수분량2','측정될수분량1','측정될수분량3'],axis=1,inplace=True)


    basic_col = test_x.columns
    kal_test = test_x

    for i in range(2,34):
        test_x['kf_X_'+str(i)]=0

    for j in range(0,len(test_x.Case.unique())): 
        print(test_x.columns)
        kal = test_x[test_x['Case'] == j]
        for i in tqdm(range(len(basic_col))):
            if((i == 0) | (i == 1)):
                continue
            current=0
            sum_c=[]
            z = kal.loc[:, kal.columns[i]]
            a = []           #필터링 된 피쳐(after)
            b = []           #필터링 전 피쳐(before)
            my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
            my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
            my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
            my_filter.H = np.array([[1.,0.]])    # Measurement function
            my_filter.P *= 1000.                 # covariance matrix
            my_filter.R = 5                      # state uncertainty
            my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) # process uncertainty   
            for k in z.values:
                my_filter.predict()
                my_filter.update(k)
                # do something with the output
                x = my_filter.x
                a.extend(x[0])
                b.append(k)
            sum_c=sum_c+a
            test_x['kf_X_'+str(i)][test_x['Case'] == j] = sum_c

    #fillna도 해줘야함

    copy_test_x = test_x.copy()

    copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)
    print(copy_test_x)

    raw_cols_te = copy_test_x.columns
    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x[copy_test_x['Case'] == i]
        mean_arr = []
        median_arr = []
        pbar = (target.columns[:])
        if(i == 1):
            print('1')
            print(pbar)
        if(i == 2):
            print('2')
            print(pbar)
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(7):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])
            for i in range(7, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-7:i])))
                median_arr.append(float(np.median(column_list[i-7:i])))
            target[f'{column}_mean_7'] = mean_arr
            target[f'{column}_median_7'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_7], axis=1)

    raw_cols_te = copy_test_x.columns

    new_test_x = pd.DataFrame()

    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x[copy_test_x['Case'] == i]
        mean_arr = []
        median_arr = []


        pbar = tqdm(target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(14):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])
                
            for i in range(14, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-14:i])))
                median_arr.append(float(np.median(column_list[i-14:i])))
            target[f'{column}_mean_14'] = mean_arr
            target[f'{column}_median_14'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_14], axis=1)

    test_x_fil = test_x.iloc[:,1:34]


    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = test_x_fil[test_x_fil['Case'] == i]
        test_merge = LPF(target,0.1,1)
        test_merge = pd.DataFrame(test_merge)
        
        new_test_x = pd.concat([new_test_x, test_merge], axis=0)
    new_test_x = new_test_x.add_suffix('_LPF')
    new_test_x = new_test_x.reset_index(drop = True)
    test_x = pd.concat([test_x,new_test_x], axis = 1)

    test_x.drop(['0_LPF'],axis = 1, inplace=True)
    test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)

    return test_x

In [ ]:
def cumsum_dataset(df1, df2):
    newdf = pd.concat([df1, df2], axis = 0)
    return newdf

In [ ]:
def make_data(input_path, fix_path, day):
    test = preprocessing_2(input_path) 

    if day != 0 : 
        test = pd.concat([test,fix_path],axis=0)
    test = test.reset_index(drop = True)

    test['05시내부온도관측치누적'] = 0
    test['19시내부온도관측치누적'] = 0
    test['23시내부온도관측치누적'] = 0

    test['05시내부습도관측치누적'] = 0
    test['19시내부습도관측치누적'] = 0
    test['23시내부습도관측치누적'] = 0

    test['05시co2관측치누적'] = 0
    test['19시co2관측치누적'] = 0
    test['23시co2관측치누적'] = 0

    test['05시ec관측치누적'] = 0
    test['19시ec관측치누적'] = 0
    test['23시ec관측치누적'] = 0

    test['05시분무량누적'] = 0
    test['19시분무량누적'] = 0
    test['23시분무량누적'] = 0

    test['05시백색광누적'] = 0
    test['19시백색광누적'] = 0
    test['23시백색광누적'] = 0

    test['05시적색광누적'] = 0
    test['19시적색광누적'] = 0
    test['23시적색광누적'] = 0

    test['05시청색광누적'] = 0
    test['19시청색광누적'] = 0
    test['23시청색광누적'] = 0

    test['05시총광량누적'] = 0
    test['19시총광량누적'] = 0
    test['23시총광량누적'] = 0

    testfilter = filtering(test)

    test = test.drop(test.filter(regex='일간누적').columns, axis=1)

    test_new = test

    test['시간대'] = 0
    test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
    test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
    test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

    test['측정될수분량2'] = 0
    for i in range(22,len(test),24):
        s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 40] = s

    test['측정될수분량1'] = 0
    for i in range(22,len(test),24):
        s = test.iloc[i+1,6] * (test.iloc[i+1,3])
        test.iloc[i, 41] = s


    test['측정될수분량3'] = 0
    for i in range(22,len(test),24):
        s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 42] = s

    ###################################################################################################################

    case_list = test['Case'].unique()

    for c in case_list : 
        if c in testfilter :
            continue
        test.loc[(test['Case'] == c ), '측정될수분량2'] = 0
        test.loc[(test['Case'] == c ), '측정될수분량1'] = 0
        test.loc[(test['Case'] == c ), '측정될수분량3'] = 0

    test_x = test
    test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()

    test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
    test_x.drop(['index'], axis = 1, inplace=True)


    t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부온도관측치'] = t

    t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t2.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부습도관측치'] = t2

    t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t3.drop(['level_1'], axis = 1, inplace = True)
    test_x['co2관측치'] = t3

    t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t4.drop(['level_1'], axis = 1, inplace = True)
    test_x['ec관측치'] = t4

    t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t5.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당분무량'] = t5

    t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t6.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당백색광량'] = t6

    t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t7.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당적색광량'] = t7

    t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t8.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당청색광량'] = t8

    t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t9.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당총광량'] = t9

    test_new_x = test_new
    test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()

    test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
    test_new_x.drop(['index'], axis = 1, inplace=True)


    for i in range(len(test_new_x)):
        #온도
        test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
        test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
        test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
        #습도
        test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
        test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
        test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
        #co2
        test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
        test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
        test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
        #ec
        test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
        test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
        test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
        #분무
        test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
        test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
        test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
        #백색
        test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
        test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
        test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
        #적색
        test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
        test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
        test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
        #청색
        test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
        test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
        test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
        #총광
        test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
        test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
        test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]



    test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)

    test_x = test_new_x

    test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)


    test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
    test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
    test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
    test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
    test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
    test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
    test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
    test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
    test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3

    test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)


    test_x['ec_x_분무05'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
    test_x['ec_x_분무19'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
    test_x['ec_x_분무23'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
    test_x['ec_x_분무평균'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)

    test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
    test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
    test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
    test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])


    test_x['수분량합'] =  test_x['측정될수분량1'] + test_x['측정될수분량2'] + test_x['측정될수분량3']

    test_x['수분량합12'] =  test_x['측정될수분량1'] + test_x['측정될수분량2']

    test_x['수분량합13'] =  test_x['측정될수분량1'] + test_x['측정될수분량3']

    test_x['수분량합23'] =  test_x['측정될수분량2'] + test_x['측정될수분량3']

    test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)

    test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)

    test_x.drop(['측정될수분량2','측정될수분량1','측정될수분량3'],axis=1,inplace=True)

    basic_col = test_x.columns
    kal_test = test_x

    for i in range(2,34):
        test_x['kf_X_'+str(i)]=0

    for j in range(0,len(test_x.Case.unique())): 
        kal = test_x
        for i in (range(len(basic_col))):
            if((i == 0) | (i == 1)):
                continue
            current=0
            sum_c=[]
            z = kal.loc[:, kal.columns[i]]
            a = []           #필터링 된 피쳐(after)
            b = []           #필터링 전 피쳐(before)
            my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
            my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
            my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
            my_filter.H = np.array([[1.,0.]])    # Measurement function
            my_filter.P *= 1000.                 # covariance matrix
            my_filter.R = 5                      # state uncertainty
            my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) # process uncertainty   
            for k in z.values:
                my_filter.predict()
                my_filter.update(k)
                x = my_filter.x
                a.extend(x[0])
                b.append(k)
            sum_c=sum_c+a
            test_x['kf_X_'+str(i)] = sum_c

    copy_test_x = test_x.copy()

    copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)

    raw_cols_te = copy_test_x.columns
    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x
        mean_arr = []
        median_arr = []

        pbar = (target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(7):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])

            for i in range(7, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-7:i])))
                median_arr.append(float(np.median(column_list[i-7:i])))
            target[f'{column}_mean_7'] = mean_arr
            target[f'{column}_median_7'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_7], axis=1)

    raw_cols_te = copy_test_x.columns



    new_test_x = pd.DataFrame()

    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x[copy_test_x['Case'] == i]
        mean_arr = []
        median_arr = []


        pbar = tqdm(target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(14):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])
                
            for i in range(14, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-14:i])))
                median_arr.append(float(np.median(column_list[i-14:i])))
            target[f'{column}_mean_14'] = mean_arr
            target[f'{column}_median_14'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_14], axis=1)

    test_x_fil = test_x.iloc[:,1:34]


    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        #target = test_x_fil[test_x_fil['Case'] == i]
        target = test_x_fil
        test_merge = LPF(target,0.1,1)
        test_merge = pd.DataFrame(test_merge)
        
        new_test_x = pd.concat([new_test_x, test_merge], axis=0)
    new_test_x = new_test_x.add_suffix('_LPF')
    new_test_x = new_test_x.reset_index(drop = True)
    test_x = pd.concat([test_x,new_test_x], axis = 1)


    test_x.drop(['0_LPF'],axis = 1, inplace=True)
    test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)
    if(day == 27):
        test_x.to_csv('last_day_fe.csv',index=False)
    return test_x

In [ ]:
def make_traindata_ctgan(input_path):
    train = pd.DataFrame()
    x = pd.read_csv(input_path)
    x = x[['obs_time','내부온도관측치','내부습도관측치','co2관측치','ec관측치','시간당분무량','시간당백색광량','시간당적색광량','시간당청색광량']]
    col_list = x.columns[1:]
    for i in range(0,28) :
        day = x.iloc[24*i:24*i+24]
        time_list = day['obs_time'].unique()
        for t in range(0,len(time_list)):
            for col in col_list:
                time = time_list[t]
                value1 = day[day['obs_time']==time][col].iloc[0]
                x[col+str(time)] = value1
        nx = x.iloc[:1,9:]
        train = pd.concat([train,nx]).reset_index(drop=True)
    return train

def make_raw (df, day) :
    df_numpy = df.values
    return_arr = []
    
    for i in range(len(df_numpy)) :
            now = df_numpy[i]
            for j in range(24):
                value = np.insert(now[j*8:j*8+8],0, int(day)) # dat
                value = np.insert(value , 1 ,int(j)) # obs_time
                value = np.insert(value , 0 ,int(i)) # case
                return_arr.append(value.tolist())

    return_df = pd.DataFrame(np.array(return_arr),columns = ['Case','DAT', 'obs_time', '내부온도관측치', '내부습도관측치','co2관측치', 
                                        'ec관측치', '시간당분무량', '시간당백색광량',  '시간당적색광량',  '시간당청색광량' ])
    
    return_df['시간당총광량'] = return_df['시간당백색광량'] + return_df['시간당적색광량'] + return_df['시간당청색광량']
    
    # 중앙값 차이 값으로 값 변환
    return_df['내부온도관측치'] =  return_df['내부온도관측치']  - 2
    return_df['시간당분무량'] = return_df['시간당분무량'] + 1251.865000
    return_df['ec관측치'] = return_df['ec관측치'] +1.7

    cumsum_list = ['일간누적분무량', '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량']
    per_time_list = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량']

    for i in range(0, 5):
        col1 = cumsum_list[i]
        col2 = per_time_list[i]
        return_df[col1] = 0
        return_df[col1] = return_df.groupby((return_df.obs_time == 0).cumsum()).agg(col2).cumsum()

    return_df = return_df[['Case', 'DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
             '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
             '일간누적청색광량', '시간당총광량', '일간누적총광량']]

    return return_df

# CTGAN

In [ ]:
# [path] 생성한 데이터샘플 저장
generated_path = "/content/drive/MyDrive/dacon/상/"

# [path] ctgan에 넣을 데이터 
input_path1 = '/content/drive/MyDrive/dacon/gan/상추/train_input/CASE_21.csv' 

# [flatten] real data -> ctgan input data
raw_data1 = make_traindata_ctgan(input_path1)

<ipython-input-10-5e425fb359fb>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  x[col+str(time)] = value1


In [ ]:
def augmentation(mode, input_path1, save_path):
    seed.seed_everything(0)

In [ ]:
def augmentation(mode, epochs, file_list, save_path):

    # seed everything
    seed.seed_everything(0)
    
    # augmentation based on ctgan
    for idx, path in enumerate(file_list):

        # data
        X = pd.read_csv(path)

        # del cumsum cols
        X = preprocessing.ctgan_preprocessing(X)
        X_pre = X.iloc[:, 2:] # except discrete_cols
        if mode == 'train':
            X_pre.to_csv(f'./data/aug_input/train/1_del_cumsum/TRAIN{idx}.csv', index=False)
        elif mode == 'test':
            X_pre.to_csv(f'./data/aug_input/test/1_del_cumsum/TEST{idx}.csv', index=False)
        print('Done. (del cumsum columns)')

        # linear
        X_pre = shape.linear(X_pre)
        if mode == 'train':
            X_pre.to_csv(f'./data/aug_input/train/2_linear/TRAIN{idx}.csv', index=False)
        elif mode == 'test':
            X_pre.to_csv(f'./data/aug_input/test/2_linear/TEST{idx}.csv', index=False)
        print('Done. (linear)')

    # groupby day
    if mode == 'train':
        linear_path = './data/aug_input/train/2_linear/'
        linear_files = listdir(linear_path)
        X_pre = shape.groupby_day('train', linear_path, linear_files)
    elif mode == 'test':
        linear_path = './data/aug_input/test/2_linear/'
        linear_files = listdir(linear_path)
        X_pre = shape.groupby_day('test', linear_path, linear_files)
    print('Done. (groupby day)')
    
    # augmentation based on ctgan
    if mode == 'train':

        for d in range(0, 28): # train
            X_pre = shape.get_groups('train', d) # X_pre.shape = (28, 193)

            # fit model
            model = CTGAN(verbose=True)
            model.fit(X_pre, epochs=epochs) # discrete_cols, epochs=epochs)

            # generate samples based on learned model
            aug_X = model.sample(50)
            aug_X.to_csv(f'./data/aug_input/train/4_aug/TRAIN{d}', index=False)
            print(aug_X)

    elif mode == 'test': 

        for d in range(0, 28):  # test
            X_pre = shape.get_groups('test', d)

            # fit model
            model = CTGAN(verbose=True)
            model.fit(X_pre, epochs=epochs) # discrete_cols, epochs=epochs)

            # generate samples based on learned model
            aug_X = model.sample(100)
            aug_X.to_csv(f'./data/aug_input/test/4_aug/TEST{d}', index=False)

    print('Done. (augmentation)')

    # create dataset
    DAT_OBS_TIME = pd.read_csv('./data/train_input/CASE_01.csv').iloc[:, :2]

    if mode == 'train':
        # create
        shape.create('train')
        # reshape
        shape.reshape('train')

        # preprocessing to input predict model
        for idx, f in enumerate(os.listdir('./data/aug_input/train/6_reshape/')):
            file_path = './data/aug_input/train/6_reshape/' + f   # file path

            target = pd.read_csv(file_path)                       # dataframe
            target = preprocessing.save_preprocessing(target)     # range cutoff
            target = cumsum.make_cumsum_cols(target)              # create cumsum cols
            target = pd.concat([DAT_OBS_TIME, target], axis=1)    # create DAT, obs_time cols
            
            target.to_csv(f'./data/aug_input/train/7_fin/TRAIN{idx}.csv', index=False) # save file

    if mode == 'test':
        return
    
    print('Done. (create)')

In [ ]:
# 일주일간격으로 넣기
model = CTGAN(epochs=100)

input_data = raw_data1.iloc[:7]
for i in range(0,7):
    model.fit(input_data)
    output_data = model.sample(50)
    dayi = make_raw(output_data,i)
    dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

input_data = raw_data1.iloc[7:14]
for i in range(7,14):
    model.fit(input_data)
    output_data = model.sample(50)
    dayi = make_raw(output_data,i)
    dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

input_data = raw_data1.iloc[14:21]
for i in range(14,21):
    model.fit(input_data)
    output_data = model.sample(50)
    dayi = make_raw(output_data,i)
    dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

input_data = raw_data1.iloc[21:]
for i in range(21,28):
    model.fit(input_data)
    output_data = model.sample(50)
    dayi = make_raw(output_data,i)
    dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

In [ ]:
gen_data = pd.read_csv('/content/drive/MyDrive/dacon/상/generate_day_0_50.csv')  

In [ ]:
gen_data_r = pd.read_csv('/content/drive/MyDrive/dacon/상/generate_day_0_50_R.csv')  

In [ ]:
!python --version

In [ ]:
dir(ctgan)

['CTGANSynthesizer',
 'TVAESynthesizer',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'data_sampler',
 'data_transformer',
 'demo',
 'load_demo',
 'synthesizers']

In [ ]:
from ctgan import CTGAN

In [ ]:
import numpy as np
import random
import os
import torch

def seed_everything(seed: int=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)               # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True      # type: ignore

''' sample '''
# seed_everything(0)

' sample '

In [ ]:
gen_data.describe()

,Case,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
count,1200.000000,1200.0,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,24.500000,0.0,11.500000,25.082561,47.597564,609.447384,2.712098,1289.343698,16079.878495,11410.406149,124976.429636,5361.976021,56884.628572,5218.788291,56646.555776,21991.170461,238507.613985
std,14.436886,0.0,6.925073,4.357251,11.521916,122.266478,0.540528,100.007346,8937.310671,8779.316974,104061.643993,5367.706385,48891.541998,4890.033061,47323.659551,17574.111248,198968.785480
min,0.000000,0.0,0.000000,17.508419,20.215086,253.974424,-0.074641,1077.672490,1116.142769,-0.003465,-0.009029,-12527.981649,-6082.824529,-12623.906178,-0.009770,-6896.694912,-0.016886
25%,12.000000,0.0,5.750000,21.094720,40.381769,519.054247,2.612218,1215.668342,8416.138706,0.000761,12791.169014,0.000995,0.006898,0.000525,1556.756809,0.001459,17861.337000
50%,24.500000,0.0,11.500000,24.590141,46.548378,596.631776,2.899892,1274.642872,16047.668034,18255.187155,117491.154908,7146.139806,49706.310932,7264.088974,50675.913353,32295.526308,218355.964078
75%,37.000000,0.0,17.250000,28.785510,53.740551,693.774218,2.930225,1354.896462,23696.867401,18255.190997,223270.217629,9763.319915,103335.276547,9242.350275,100540.963283,37260.860023,433730.709186
max,49.000000,0.0,23.000000,35.166724,78.068695,1034.514649,4.301894,1538.416208,31759.402625,18976.418652,274787.832527,20617.725455,155466.978689,18541.184447,147168.803730,49951.936480,558598.599849


In [ ]:
gen_data

,Case,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0.0,0.0,0.0,20.740200,62.332033,630.782162,1.992532,1289.950544,1289.950544,0.001689,0.001689,-0.002890,-0.002890,0.001840,0.001840,0.000639,0.000639
1,0.0,0.0,1.0,20.229342,46.310652,780.852077,2.473932,1312.642895,2602.593440,-0.000577,0.001112,-0.000128,-0.003018,-0.001217,0.000622,-0.001922,-0.001284
2,0.0,0.0,2.0,20.043591,65.829749,473.661565,2.449282,1237.884846,3840.478286,-0.002314,-0.001202,-0.000681,-0.003699,-0.002042,-0.001419,-0.005037,-0.006321
3,0.0,0.0,3.0,19.306933,58.943321,776.222156,3.130211,1463.623044,5304.101330,0.001609,0.000406,-0.000761,-0.004460,0.000156,-0.001264,0.001003,-0.005317
4,0.0,0.0,4.0,21.807347,59.716630,451.788838,2.649863,1164.036294,6468.137624,-0.000383,0.000023,-0.001830,-0.006290,0.002959,0.001695,0.000745,-0.004572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,49.0,0.0,19.0,28.156704,41.630764,847.183838,2.930589,1296.297330,25858.428469,18255.191613,255322.209341,9763.319096,125748.910623,9242.348754,122821.082220,37260.859464,503892.202184
1196,49.0,0.0,20.0,27.330696,52.005094,720.800619,2.933938,1468.481444,27326.909912,18255.187284,273577.396625,9763.318693,135512.229316,9242.351671,132063.433891,37260.857648,541153.059832
1197,49.0,0.0,21.0,22.601852,46.457872,514.417395,2.899231,1259.595065,28586.504977,682.571673,274259.968298,285.289950,135797.519266,678.343682,132741.777573,1646.205305,542799.265138
1198,49.0,0.0,22.0,20.650645,66.866570,521.069441,2.904725,1175.062902,29761.567879,0.002655,274259.970954,-0.001965,135797.517301,0.002500,132741.780073,0.003190,542799.268328


In [ ]:
gen_dataf = pd.read_csv('/content/drive/MyDrive/dacon/상/generate_day_0_50_R_R2.csv')  
gen_dataf

,Case,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0.0,0.0,0.0,19.619913,62.134115,506.539955,2.343749,1211.628084,1211.628084,0.001211,0.001211,-0.000456,-0.000456,-0.002415,-0.002415,-0.001660,-0.001660
1,0.0,0.0,1.0,19.111524,60.328817,506.262800,3.259866,1238.024427,2449.652511,0.000327,0.001537,0.002060,0.001604,-0.001341,-0.003756,0.001046,-0.000614
2,0.0,0.0,2.0,21.580406,52.188986,448.831373,2.629285,1297.977489,3747.630000,-0.003435,-0.001898,-0.001400,0.000204,0.000037,-0.003719,-0.004799,-0.005413
3,0.0,0.0,3.0,18.663138,55.140220,536.459242,3.020284,1285.371350,5033.001350,-0.002555,-0.004453,0.000320,0.000523,-0.001635,-0.005354,-0.003870,-0.009283
4,0.0,0.0,4.0,17.853873,70.340961,426.924704,3.550892,1180.334276,6213.335626,-0.002734,-0.007187,-0.002458,-0.001934,-0.000907,-0.006261,-0.006099,-0.015382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,49.0,0.0,19.0,23.418023,42.319488,551.446837,2.894032,1206.757389,25657.051804,18255.192898,254546.902663,9763.321749,157061.615266,9242.352073,105309.179849,37260.866720,516917.697778
1196,49.0,0.0,20.0,24.755485,45.468801,604.912222,2.909392,1275.828564,26932.880369,18255.192094,272802.094757,9763.317942,166824.933208,9242.348169,114551.528018,37260.858205,554178.555983
1197,49.0,0.0,21.0,21.283085,43.953098,672.838663,2.918389,1390.259041,28323.139409,473.556836,273275.651593,622.319326,167447.252534,100.468774,114651.996791,1196.344936,555374.900919
1198,49.0,0.0,22.0,19.745678,63.749332,738.778261,2.908017,1226.060296,29549.199705,0.000937,273275.652530,0.001807,167447.254341,0.001998,114651.998789,0.004742,555374.905661


In [ ]:
gen_dataf.describe()

,Case,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
count,1200.000000,1200.0,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,24.500000,0.0,11.500000,23.919908,49.625333,593.201097,2.835704,1296.895197,16174.229896,11386.074952,124804.301593,6251.983126,69634.440383,5133.960440,53350.416168,22772.018518,247789.158145
std,14.436886,0.0,6.925073,3.927602,10.928158,131.289814,0.495922,98.254746,8992.819374,8781.241056,103906.880845,5269.311969,57227.421632,4986.391377,47762.048750,17908.294083,207822.382733
min,0.000000,0.0,0.000000,17.180361,19.818557,274.223003,0.054645,1087.354350,1207.431680,-0.003551,-0.011805,-8926.440690,-0.010980,-10904.723341,-10904.724485,-0.008376,-0.023299
25%,12.000000,0.0,5.750000,20.598272,41.986005,490.443618,2.774582,1225.985710,8539.125637,0.000753,12731.381509,0.000850,3969.476025,0.000208,0.000942,0.000746,12187.369903
50%,24.500000,0.0,11.500000,23.341594,48.158864,582.350793,2.901226,1284.234776,16295.744765,18228.806898,117482.256500,9436.853191,66988.921826,7076.117975,44223.133856,35094.142343,230037.022115
75%,37.000000,0.0,17.250000,26.885022,56.613016,682.403417,2.976970,1366.853973,24229.722336,18255.190055,222786.923215,9788.086828,125016.157173,9242.350725,98115.448696,37260.862505,452264.297204
max,49.000000,0.0,23.000000,34.619006,77.715392,1046.751362,4.496499,1557.985056,32337.989092,18545.240480,273945.216177,20371.903161,167447.255086,19294.070807,145836.377956,50012.912970,572064.550975


In [ ]:
# 일주일간격으로 넣기
model = CTGAN(epochs=100)

input_data = raw_data1.iloc[:7]
for i in range(0,1):
    model.fit(input_data)
    output_data = model.sample(50)
    dayi = make_raw(output_data,i)
    dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R_R2.csv'), index=False)

/usr/local/lib/python3.8/dist-packages/ctgan/data_transformer.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/usr/local/lib/python3.8/dist-packages/ctgan/data_transformer.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/usr/local/lib/python3.8/dist-packages/ctgan/data_transformer.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [ ]:
# 일주일간격으로 넣기
model = CTGAN(epochs=100)

input_data = raw_data1.iloc[:7]
for i in range(0,1):
    model.fit(input_data)
    output_data = model.sample(50)
    dayi = make_raw(output_data,i)
    dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R_R.csv'), index=False)

/usr/local/lib/python3.8/dist-packages/ctgan/data_transformer.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/usr/local/lib/python3.8/dist-packages/ctgan/data_transformer.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/usr/local/lib/python3.8/dist-packages/ctgan/data_transformer.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [ ]:
gen_data_r

,Case,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0.0,0.0,0.0,20.514954,53.064974,578.000000,2.946695,1282.746296,1282.746296,0.00,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,1.0,20.349706,51.403426,671.433333,2.450625,1265.153913,2547.900209,0.00,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,2.0,20.025047,59.585038,664.000000,2.944704,1287.113745,3835.013954,0.00,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,3.0,20.481704,68.025000,655.866667,2.952886,1377.865000,5212.878954,0.00,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,4.0,21.118333,57.341041,647.000000,2.954373,1251.865000,6464.743954,0.00,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,49.0,0.0,19.0,28.943333,46.390442,773.000000,2.898044,1251.865000,25922.582387,18255.19,255134.347635,9763.3200,110439.59516,9242.350000,122342.031915,37260.860000,487915.974710
1196,49.0,0.0,20.0,27.026667,43.369121,549.000000,2.893534,1251.865000,27174.447387,18255.19,273389.537635,9763.3200,120202.91516,9242.350000,131584.381915,37260.860000,525176.834710
1197,49.0,0.0,21.0,21.254883,47.749745,630.984708,2.905819,1289.369076,28463.816463,529.00,273918.537635,519.6072,120722.52236,360.287006,131944.668921,1408.894206,526585.728915
1198,49.0,0.0,22.0,21.054498,54.412245,568.589291,2.924859,1261.025760,29724.842223,0.00,273918.537635,0.0000,120722.52236,0.000000,131944.668921,0.000000,526585.728915


In [ ]:
gen_data = pd.read_csv('/content/drive/MyDrive/dacon/상/generate_day_0_50_R_R.csv')  
gen_data

,Case,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0.0,0.0,0.0,20.620653,60.497467,511.000000,2.946695,1251.865000,1251.865000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,1.0,20.373816,66.035000,671.433333,2.949120,1251.865000,2503.730000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,2.0,20.483061,67.196667,664.000000,2.950995,1309.985375,3813.715375,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,3.0,19.751064,57.879349,655.866667,2.532093,1377.865000,5191.580375,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,4.0,20.080734,68.698333,503.000000,2.628418,1314.865000,6506.445375,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,49.0,0.0,19.0,25.766329,45.453699,535.000000,2.910408,1251.865000,26153.680861,18255.19,254897.634379,9763.32,116914.937718,9242.350000,114516.344688,37260.860000,486328.916785
1196,49.0,0.0,20.0,27.026667,48.605000,687.000000,2.893534,1251.865000,27405.545861,18255.19,273152.824379,9763.32,126678.257718,9242.350000,123758.694688,37260.860000,523589.776785
1197,49.0,0.0,21.0,21.113298,57.767118,561.194839,2.914143,1336.361114,28741.906975,683.00,273835.824379,165.48,126843.737718,392.765084,124151.459772,1241.245084,524831.021869
1198,49.0,0.0,22.0,20.239232,63.601667,695.150000,2.924534,1287.878599,30029.785575,0.00,273835.824379,0.00,126843.737718,0.000000,124151.459772,0.000000,524831.021869


In [ ]:
# 예측모델 로드
predict_model = TabularPredictor.load("/content/drive/MyDrive/Project_L/1등모델", require_version_match=False)

# 0일차 예측
gen_data = pd.read_csv('/content/drive/MyDrive/dacon/상/generate_day_0_50.csv')  
max_list = []
for i in range(50):
    gen_data_new = gen_data[i*24:(i+1)*24]
    gen_data_new['Case']=0
    gan_data = makedatasets(gen_data_new)
    y_pred = predict_model.predict(gan_data)
    max_list.append(y_pred.max())
max_list = pd.DataFrame(max_list)
idx_max = max_list.idxmax()
gen_dat = gen_data[int(idx_max.values)*24:(int(idx_max.values)+1)*24]
gen_dat['DAT'] = 0
print(max_list.max())
# fix data 경로
gen_dat.to_csv("/content/drive/MyDrive/dacon/fix_ver2.csv", index=False)

In [ ]:
predict_model = TabularPredictor.load("/content/drive/MyDrive/Project_L/1등모델", require_version_match=False)

for day in range(1, 28):
    max_0 = pd.read_csv("/content/drive/MyDrive/dacon/fix_ver2.csv")
    print(f'{day}일차 Start')
    gen_data = pd.read_csv(f'/content/drive/MyDrive/dacon/상/generate_day_{day}_50.csv')  # 동적변경
    max_list = []
    for i in range(50):
        gen_data_new = gen_data[i*24:(i+1)*24]
        gen_data_new = gen_data_new.reset_index(drop=True)
        max_0['Case'] = i
        gen_data_new['Case'] = i
        gen_data_new['DAT'] = day 
        gan_data = make_data(gen_data_new, max_0, day)
        y_pred = predict_model.predict(gan_data)
        max_list.append(y_pred.max())
    max_list = pd.DataFrame(max_list)
    print(max_list.max())
    idx_max = max_list.idxmax()
    gen_dat = gen_data[int(idx_max.values)*24:(int(idx_max.values)+1)*24]
    gen_dat['DAT'] = day
    con_dat = pd.concat([max_0, gen_dat], axis=0)
    con_dat.to_csv("/content/drive/MyDrive/dacon/fix_ver2.csv", index=False)
    print(f'{day}일차 End------------------------------')

1일차 Start


100%|██████████| 102/102 [00:00<00:00, 1535.72it/s]


0    7.673206
dtype: float32
1일차 End------------------------------
2일차 Start


100%|██████████| 102/102 [00:00<00:00, 1519.55it/s]


0    9.133521
dtype: float32
2일차 End------------------------------
3일차 Start


...

0    160.709732
dtype: float32
25일차 End------------------------------
26일차 Start


100%|██████████| 102/102 [00:00<00:00, 1453.33it/s]


0    161.349014
dtype: float32
26일차 End------------------------------
27일차 Start


100%|██████████| 102/102 [00:00<00:00, 1423.32it/s]


0    161.349014
dtype: float32
27일차 End------------------------------


In [ ]:
Y_PRED = [6.492403, 7.673206, 9.133521, 10.28068, 15.817433,  21.04789 ,23.993586, 25.401606, 32.695137, 41.393791, 
          58.98909, 70.797424, 80.979195, 106.896126, 111.689438, 117.851593, 124.746178, 133.241379, 140.985886,
          146.298294, 151.695557, 154.781738, 156.817123, 157.856873, 158.730499, 160.709732, 161.349014, 161.349014  ]